Why did my answer differ from Ben's example ?


In [244]:
docs = ["The cat sat on my head", "The dog sat on my bed"]

# TF: how often a given word appears within a document

# Example: TF(doc1)
#{'The': 0.16666666666666666,
# 'cat': 0.16666666666666666,
# 'bed': 0.0,
# 'dog': 0.0,
# 'my': 0.16666666666666666,
# 'head': 0.16666666666666666,
# 'sat': 0.16666666666666666,
# 'on': 0.16666666666666666}

# IDF: downscales words that appear a lot across documents
# idf(t, D) = log (<total number of documents in the corpus D>   /  
# <number of documents in D where the term t appears at least once>)
#                     100             3           1

## I think this is from from wiki: https://en.wikipedia.org/wiki/Tf%E2%80%93idf
## idf(t, D) = {\displaystyle \mathrm {tf} (t,d)=0.5+0.5\cdot {\frac {f_{t,d}}{\max\{f_{t',d}:t'\in d\}}}}
## {idf} (t,D)=\log {\frac {N}{|\{d\in D:t\in d\}|}}} \mathrm{idf}(t, D) = \log \frac{N}{|\{d \in D: t \in d\}|}
    
# Example: 
# {'on': 0.0, 'dog': 0.3010299956639812, 'bed': 0.3010299956639812, 'face': 0.3010299956639812, 
#  'The': 0.0, 'cat': 0.3010299956639812, 'my': 0.0, 'sat': 0.0}
#
#
# tf-idf = TF * IDF 
# Example:
# {'on': 0.0, 'dog': 0.0, 'bed': 0.0, 'cat': 0.050171665943996864,
#   'The': 0.0, 'face': 0.050171665943996864, 'my': 0.0, 'sat': 0.0}



In [251]:
# my implemntation from iview

from collections import Counter
from math import log

def compute_idf(corpora):
    c = Counter()
    num_doc = len(corpora)
    
    for doc in corpora:
        t = set(doc)
        c += Counter(t)  # counter overrides +
    
    for k in c.keys():
        c[k] = log ( num_doc / c[k]) 
        
    return c

def compute_tf(terms, idf):
    c = Counter(terms)    
    for k in c.keys():
        c[k] *= idf[k] * 1/len(terms) 
    return c

def tokenize(body):
    return body.lower().split()  



In [252]:
idf = compute_idf(list(map(tokenize,[doc1, doc2])))

# Example: 
# {'on': 0.0, 'dog': 0.3010299956639812, 'bed': 0.3010299956639812, 'face': 0.3010299956639812, 
#  'The': 0.0, 'cat': 0.3010299956639812, 'my': 0.0, 'sat': 0.0}
# vs
#
#
#
print (dict(idf))
# {'the': 0.0, 'head': 0.6931471805599453, 'cat': 0.6931471805599453, 'on': 
# 0.0, 'my': 0.0, 'sat': 0.0, 'bed': 0.6931471805599453, 'dog': 0.6931471805599453}

# We get different a different IDF vector
# it's off by a constant factor... 


{'the': 0.0, 'head': 0.6931471805599453, 'cat': 0.6931471805599453, 'on': 0.0, 'my': 0.0, 'sat': 0.0, 'bed': 0.6931471805599453, 'dog': 0.6931471805599453}


In [261]:
# where does 0.3010299956639812 even come from ?
# it must be a low integer fraction of some sort, maybe with a smoothing term ?

from math import log, exp

needle = 0.3010299956639812 

def close(a, b, eps=10**-8):
    return abs(float(a) - float(b)) < eps
K = 1000


# loop through all low integer fractions...
for i in range(1,K):
    for j in range(i,K):
        if close( log(float(i)/ j), needle):
            print (f"case a: {i}/{j}")
        if close( log(float(i)/ j + 1), needle):
            print (f"case b: {i}/{j}")
        if close( (float(i)/ j), needle):
            print (f"case c: {i}/{j}")
            



In [258]:
# nothing !
# oh, wait, 0.3010299956639812 = log10(2)  !!
log(2,10)

0.30102999566398114

In [262]:
E = exp(1)
factor = log(E, 10)
print ({ k: v * factor for k,v in idf.items() })  # next time use numpy this is dumb

# {'the': 0.0, 'head': 0.30102999566398114, 'cat': 0.30102999566398114, 'on': 0.0, 'my': 0.0, 
# 'sat': 0.0, 'bed': 0.30102999566398114, 'dog': 0.30102999566398114}

# yay, that explains it. He used log10



{'the': 0.43429448190325176, 'head': 0.735324477567233, 'cat': 0.735324477567233, 'on': 0.43429448190325176, 'my': 0.43429448190325176, 'sat': 0.43429448190325176, 'bed': 0.735324477567233, 'dog': 0.735324477567233}


In [196]:
# am I consistent with sklearn ?

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

tokenized = [word_tokenize(d.lower()) for d in docs]
cv = CountVectorizer()
cv.fit(docs)         

counts = cv.transform(docs) # xform to sparse matrix
counts /= counts.sum(axis=1)
vals = TfidfTransformer(norm=None, smooth_idf=False).fit_transform(counts).todense()


In [259]:
# matching sklearn's smoothing formualtion adding on the smooothing parameter

def compute_idf_smoothed(corpora):
    c = Counter()
    num_doc = len(corpora)
    
    for doc in corpora:
        t = set(doc)
        c += Counter(t) 
        
    for k in c.keys():
        c[k] = log ( num_doc / c[k]) 
        c[k] += 1        # to match sklearn
    return c

In [260]:

print ("sklearn")
print (dict(zip(cv.get_feature_names(), vals[0].tolist()[0])))
print (dict(zip(cv.get_feature_names(), vals[1].tolist()[0])))
print()

print ("mine")
idf = compute_idf_smoothed(list(map(tokenize,[doc1, doc2])))
print(compute_tf(tokenize(doc1), idf))
print(compute_tf(tokenize(doc2), idf))
print ()


sklearn
{'bed': 0.0, 'cat': 0.2821911967599909, 'dog': 0.0, 'head': 0.2821911967599909, 'my': 0.16666666666666666, 'on': 0.16666666666666666, 'sat': 0.16666666666666666, 'the': 0.16666666666666666}
{'bed': 0.2821911967599909, 'cat': 0.0, 'dog': 0.2821911967599909, 'head': 0.0, 'my': 0.16666666666666666, 'on': 0.16666666666666666, 'sat': 0.16666666666666666, 'the': 0.16666666666666666}

mine
Counter({'cat': 0.2821911967599909, 'head': 0.2821911967599909, 'the': 0.16666666666666666, 'sat': 0.16666666666666666, 'on': 0.16666666666666666, 'my': 0.16666666666666666})
Counter({'dog': 0.2821911967599909, 'bed': 0.2821911967599909, 'the': 0.16666666666666666, 'sat': 0.16666666666666666, 'on': 0.16666666666666666, 'my': 0.16666666666666666})

